In [ ]:
!pip install scikit-image matplotlib tqdm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import blob_dog
from itertools import product
from tqdm import tqdm
from read_chroma import read_chromato_and_chromato_cube
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import read_chroma
import baseline_correction
import projection
import plot
import peak_detection

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'

mod_time = 1.7

In [ ]:
chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)

In [ ]:

min_sigma = 2
max_sigma = 30
sigma_ratio = 1.6
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0]
mass_isoprene = 67 - 35
mass_ethanol =  46 - 35
 
mass = mass_ethanol

for thresh in thresholds:
    coordinates_all_mass = []
    print("thresh", thresh)
    

    coord = peak_detection.blob_dog_kernel(
            None, m_chromato=chromato_cube_preprocessed[mass], 
            min_sigma=min_sigma, max_sigma=max_sigma, seuil=None,
            threshold_abs=thresh,
            sigma_ratio=sigma_ratio)
    
    
    if len(coord) == 0:
        print(f"No coordinates found for threshold {thresh}. Skipping to next threshold.")
        continue
    
    for t1, t2, r in coord:
        coordinates_all_mass.append([mass, t1, t2])

    if len(coordinates_all_mass) == 0:
        print(f"No valid coordinates extracted for threshold {thresh}. Skipping to next threshold.")
        continue

    # coordinates_all_mass = np.delete(coordinates_all_mass, 0 , -1)
     # Use a safer approach to create a new array without the first column
    coordinates_without_mass = np.array([coord[1:] for coord in coordinates_all_mass])
    
    blobs_scaled = StandardScaler().fit_transform(coordinates_without_mass)
    # blobs_scaled = blobs
    clustering = DBSCAN(eps=0.2, min_samples=5).fit(blobs_scaled)

    n_clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
# initialisation de la liste des clusters
    clusters = [[] for _ in range(n_clusters)]

    for i, (t1, t2) in enumerate(coordinates_without_mass):
        label = clustering.labels_[i]
        if label != -1: # enleve les points de bruit
            clusters[label].append([t1, t2])

    print("nb de clusters", len(clusters))
    n_noise = np.sum(clustering.labels_ == -1)
    print(f"Nombre de points considérés comme du bruit : {n_noise}")


    coordinates = []
    non_empty_clusters = [cluster for cluster in clusters if len(cluster) > 0]
    if not non_empty_clusters:
        continue
    
    for cluster in clusters:
        if not cluster:
            print("Erreur, cluster vide")
            break
            
        elif (len(cluster) > 1):
            # point le + intense:
            # coord = cluster[np.argmax(np.array([chromato[coord[0], coord[1]] for coord in cluster]))] 

            arr = np.array(cluster)
            # centre geometrique du cluster
            median_x = np.median(arr[:, 0]) 
            median_y = np.median(arr[:, 1])
            coord=[round(median_x),round(median_y)]
        else:
            coord = cluster[0]

        coordinates.append(coord)

    peaks_coordinates = np.array(coordinates, dtype=float)


    tic_chromato, time_rn, spectra_obj = read_chroma.read_chroma(file_path, mod_time, None)
    chromato_tic_preprocessed = baseline_correction.chromato_reduced_noise(tic_chromato)

    coordinates_in_chromato = projection.matrix_to_chromato(
        peaks_coordinates, 
        time_rn, 
        mod_time, 
        chromato_tic_preprocessed.shape
        )
    plot.visualizer(
        (chromato_tic_preprocessed, time_rn),
        mod_time,
        log_chromato=False,
        points=coordinates_in_chromato)
        
    



In [ ]:


# Paramètres à tester
min_sigma_list = [1, 2, 3]
max_sigma_list = [10, 20]
threshold_list = [0.01, 0.03, 0.05]
masse_isoprene = mass = 67


results = []

# Grid Search
for min_sigma, max_sigma, threshold in tqdm(product(min_sigma_list, max_sigma_list, threshold_list),
                                            total=len(min_sigma_list)*len(max_sigma_list)*len(threshold_list)):
    
    blobs = blob_dog(chromato_cube_preprocessed[mass],
                    min_sigma=min_sigma,
                    max_sigma=max_sigma,
                    threshold=threshold,
                    overlap=0.5)

    score = len(blobs)  # Critère simple : plus de blobs = plus de pics détectés

    results.append({
        'min_sigma': min_sigma,
        'max_sigma': max_sigma,
        'threshold': threshold,
        'score': score,
        'blobs': blobs
    })

# Tri des meilleurs paramètres (par exemple : le plus de blobs)
best_result = sorted(results, key=lambda x: x['score'], reverse=True)[0]

print("\n✅ Best Parameters Found:")
print(f"min_sigma = {best_result['min_sigma']}")
print(f"max_sigma = {best_result['max_sigma']}")
print(f"threshold = {best_result['threshold']}")
print(f"Nb blobs detected = {best_result['score']}")

# Affichage des blobs détectés
fig, ax = plt.subplots()
ax.imshow(file_path, cmap='gray')
for blob in best_result['blobs']:
    y, x, r = blob
    c = plt.Circle((x, y), r * np.sqrt(2), color='red', linewidth=1.5, fill=False)
    ax.add_patch(c)
plt.title("Best blob_dog parameters")
plt.show()



overlap: par defaut = 0.5
sert à supprimer les blobs redondants qui se chevauchent trop.
Lorsque plusieurs blobs détectés se recouvrent de plus de overlap (par défaut 0.5), seul le plus significatif est gardé (en général celui avec le rayon le plus grand ou la valeur de réponse la plus élevée).


In [ ]:
min_sigma = 1
max_sigma = 30
sigma_ratio = 1.6
threshold_abs = 0.01

In [ ]:
from skimage.feature import blob_log
import matplotlib.pyplot as plt

#overlap
def test_overlap(chromato_2d, min_sigma, max_sigma, num_sigma, threshold_abs):
    overlaps = [0.1, 0.3, 0.5, 0.7, 0.9]
    results = {}

    for ov in overlaps:
        blobs = blob_log(
            chromato_2d,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            num_sigma=num_sigma,
            threshold=threshold_abs,
            overlap=ov
        )
        results[ov] = blobs
        print(f"Overlap {ov} : {len(blobs)} blobs")

    return results